## Tensorflow
- `Tensorflow`가 예전에는 쓰기 어려운 모델이었음 (코딩할 줄 아는 사람들만 사용)
- 그래서 `pytorch`가 많이 쓰이다 보니, `Tensorflow`에서도 쉽게 사용할 수 있는 `Keras` 만듦
- `Tensorflow 2.0`에서는 `keras`와 합쳐진 `tf.keras.Model`이나 `Sequential` 많이 사용
- `Tensorflow`에서 train step, test step을 사용하는 class 구조는 `pytorch lightening`과 비슷
  - `pytorch lightening` : `pytorch`를 더 쉽게 사용하기 위한 library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/#fastcampus')
drive_project_root = '/content/drive/MyDrive/#fastcampus'
# !pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

In [ ]:
from datetime import datetime

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf

GPU 확인

In [ ]:
tf.config.list_physical_devices()

In [ ]:
!nvidia-smi

https://www.tensorflow.org/</br>
- https://www.tensorflow.org/overview/?hl=ko</br>
- 튜토리얼 : https://www.tensorflow.org/tutorials?hl=ko
- API > Tensorflow : 각 함수에 대한 설명
  - 구글에 'Tensorflow API 한글' 검색하면 번역본도 볼 수 있음

초보자용 vs 전문가용
- 수업에서는 전문가용으로 할 예정
- 초보자용에서 사용하는 Sequential 버전(순차적으로 build 하는 방법)에는 한계가 있기 때문
- 실제 현업/연구에서는 Sequential 거의 안 씀

## define gpu
- https://www.tensorflow.org/api_docs/python/tf/distribute/MirroredStrategy
- This strategy is typically used for training on one machine with multiple GPUs.
- 아래 코드 결과 보면 GPU 0번 잡아서 가져옴

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

## data, data loader 정의

- 사실 tensorflow에서는 data loader를 정의를 안 하기도 함
- dataset으로 그냥 처리 가능
- 단, 여기서는 pytorch 방식과 비교하기 위해 사용함

In [ ]:
with mirrored_strategy.scope():

    # dataset 정의 =====================================================================
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

    # normalization
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    # train/val splits
    train_size = int(len(x_train)*0.9)
    val_size = len(x_train) - train_size

    # train, test dataset 정의
    dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=1024)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(buffer_size=1024)

    # train dataset을 train과 validation으로 나누기
    # tensorflow에서는 아래와 같이 take, skip 사용해서 데이터 많이 나눔
    train_dataset = dataset.take(train_size)   # train은 dataset에서 train_size만큼 take하고
    val_dataset = dataset.skip(train_size)     # val은 전체 dataset에서 train_size만큼 skip하고 남은 것

    # 검증
    print(f'train total : {len(dataset)} (train : {len(train_dataset)}, validation : {len(val_dataset)})')
    print(f'test : {len(test_dataset)}')

    # dataloader 정의 ==================================================================
    train_batch_size = 100
    val_batch_size = 10
    test_batch_size = 100

    # drop_remainder=True : memory size가 안 맞으면 error 나는 것 방지 (pytorch는 이런 것 자동으로 처리함 = tensorflow와 차이점)
    train_dataloader = train_dataset.batch(train_batch_size, drop_remainder=True)
    val_dataloader = val_dataset.batch(val_batch_size, drop_remainder=True)
    test_dataloader = test_dataset.batch(test_batch_size, drop_remainder=True)

In [ ]:
sample_example = next(iter(train_dataloader))
print(sample_example)

## plot figure

In [ ]:
plt.figure(figsize=(10,10))
for c in range(16):
    plt.subplot(4, 4, c+1)
    plt.imshow(x_train[c].reshape(28,28), cmap='gray')
plt.show()

## make model

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self, input_dim: int, h1_dim: int, h2_dim: int, out_dim: int):
        super().__init__()

        # tensorflow는 pytorch와 다르게 flatten을 하지 않아도 되지만
        # pytorch와 비슷한 구조로 코딩하기 위해 여기서는 썼음
        self.flatten = tf.keras.layers.Flatten()

        # tf nn module vs keras module
        # 1) nn module : tensorflow 1.0에서 사용, 기능이 조금 더 많음
        # 2) keras : tensorflow 2.0에서 사용
        self.linear1 = tf.keras.layers.Dense(input_dim=input_dim, units=h1_dim)
        # self.linear2 = tf.keras.layers.Dense(input_dim=h1_dim, units=h2_dim)
        # -> 이렇게 써도 되지만, pytorh보다 keras는 flexibility가 있어서 input_dim 생략해도 알아서 인지함
        self.linear2 = tf.keras.layers.Dense(units=h2_dim)
        self.linear3 = tf.keras.layers.Dense(units=out_dim)
        self.relu = tf.nn.relu
    
    # tensorflow에서는 'training=Fasle' 구문 꼭 넣기를 권장함
    # 나중에 regularization에서 drop out 할 때 이 부분을 조절할 수 있어야 함
    # - 학습일 때는 켜고, evaluation 때는 끄고
    def call(self, input, training=False):
        x = self.flatten(input)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        out = self.linear3(x)
        out = tf.nn.softmax(out)  # output을 확률값으로 바꿈
        return out
    
    # GradientTape() 구현
    # 따로 구현해도 되지만 class 안에 이렇게 넣어주면 나중에 더 코드가 깔끔해짐
    def train_step(self, data):
        # pass
        images, labels = data
        
        with tf.GradientTape() as tape:
            outputs = self(images, training=True)
            preds = tf.argmax(outputs, 1)

            # 위에서 out이 softmax 안 거친 경우, 여기 넣을 때 softmax 처리 해줘야 함
            loss = self.compiled_loss(
                labels, outputs
            )

        # compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        logs.update({"loss": loss})
        return logs
    
    def test_step(self, data):
        # pass
        images, labels = data
        outputs = self(images, training=False)
        preds = tf.argmax(outputs, 1)
        loss = self.compiled_loss(
            labels, outputs
        )

        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        logs.update({"test_loss": loss})
        return logs

## define model

In [ ]:
n_class = 10
max_epoch = 50

with mirrored_strategy.scope():
    # channel : rgb가 없고 gray니까 1
    model = MLP(28*28*1, 128, 64, n_class)  # *args
    model_name = type(model).__name__       # MLP

    # define loss
    loss_function = tf.losses.SparseCategoricalCrossentropy()

    # define optimizer
    lr = 1e-3
    optimizer = tf.optimizers.Adam(learning_rate=lr)

    model.compile(
        loss = loss_function,
        optimizer = optimizer,
        metrics = [tf.keras.metrics.Accuracy()],
    )

    # model build
    # 이 부분 생략해도 되지만 build를 해 놓으면 나중에 debugging하기 좋음 -> 권장
    # batch 1 : 임의로 설정
    model.build((1, 28*28*1))

# 만약 build 안 하고 summary 하면 build, fit을 하거나 input shape를 넣으라고 경고 뜸
# fit은 학습이기 때문에 무거운 감이 있고 빠르게 하기 위해 build 선호
model.summary()

## define logging & callbacks

In [ ]:
log_interval = 100
run_name = f'{datetime.now()}-{model.name}'

run_dirname = 'dnn-tutorial-fashion-mnist-runs-tf'

# 경로에 'run'이라는 폴더를 만들고, run_dirname에 run_name 생성
log_dir = os.path.join(drive_project_root, "runs", run_dirname, run_name)

In [ ]:
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir, update_freq=log_interval
)

경로 잘 찾고 있는지 확인

In [ ]:
! ls /content/drive/MyDrive/\#fastcampus/runs/

In [ ]:
# tensorboard load하기 : load extension
%load_ext tensorboard

# 경로 지정 : terminal 문법이기 때문에 #을 # 그대로 인지하려면 앞에 '\' 써줘야 함
%tensorboard --logdir /content/drive/MyDrive/\#fastcampus/runs/

model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=max_epoch,
    callbacks=[tb_callback] # callback은 여러 개를 넣을 수 있기 때문에 list 형태로 지정함
)

## model testing

In [ ]:
model.evaluate(test_dataloader)

auc curve

In [ ]:
test_labels_list = []
test_preds_list = []
test_outputs_list = []

for i, (test_images, test_labels) in enumerate(tqdm(test_dataloader, position=0, leave=True, desc='testing')):
    with mirrored_strategy.scope():
        test_outputs = model(test_images)
    test_preds = tf.argmax(test_outputs, 1)

    final_outs = test_outputs.numpy()
    test_outputs_list.extend(final_outs)

    test_preds_list.extend(test_preds.numpy())
    test_labels_list.extend(test_labels.numpy())

test_preds_list = np.array(test_preds_list)
test_labels_list = np.array(test_labels_list)

test_accuracy = np.mean(test_preds_list == test_labels_list)
print(f'\nacc: {test_accuracy*100}%')

roc curve

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr = {}
tpr = {}
thresh = {}
n_class = 10

for i in range(n_class):
    fpr[i], tpr[i], thresh[i] = roc_curve(test_labels_list, np.array(test_outputs_list)[:, i], pos_label=i)

In [ ]:
fpr

plot

In [ ]:
for i in range(n_class):
    plt.plot(fpr[i], tpr[i], linestyle="--", label=f"Class {i} vs Rest")

plt.title("Multi-class ROC Curve")
plt.xlabel("Flase Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="best")
plt.show()

auc score
- multi class이기 때문에 multi_class, average option 안 넣어주면 error 발생

In [ ]:
auc_score = roc_auc_score(test_labels_list, test_outputs_list, multi_class="ovo", average="macro")

In [ ]:
print(f'auc score : {auc_score*100}')